In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingRegressor, VotingClassifier, StackingClassifier

import os
os.environ["OMP_NUM_THREADS"] = "1"

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold, cross_val_predict
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score, recall_score, roc_auc_score
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, RandomForestClassifier

import matplotlib.pyplot as plt

import time

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

import time
from xgboost import XGBClassifier

In [2]:
train = pd.read_csv('train_classification.csv')
test = pd.read_csv('test_classification.csv')

In [3]:

## turned y value numeric
most_frequent_value = train['host_is_superhost'].mode()[0]
train['host_is_superhost'].fillna(most_frequent_value, inplace=True)

train['host_is_superhost'] = train['host_is_superhost'].replace({'f': 0, 't': 1})

## turned min_nights numeric
train.minimum_minimum_nights = train.minimum_minimum_nights.astype(float)
test.minimum_minimum_nights = test.minimum_minimum_nights.astype(float)

## filled in any missing values for bed
train.beds = train.beds.bfill()
test.beds = test.beds.bfill()

## tip from professor


##important value -- used the average
train['review_scores_value'] = train['review_scores_value'].fillna(train.review_scores_value.mean())
test['review_scores_value'] = test['review_scores_value'].fillna(test.review_scores_value.mean())




most_frequent_value = train['review_scores_location'].mode()[0]
train['review_scores_location'].fillna(most_frequent_value, inplace=True)

most_frequent_value = test['review_scores_location'].mode()[0]
test['review_scores_location'].fillna(most_frequent_value, inplace=True)


most_frequent_value = train['host_response_rate'].mode()[0]
train['host_response_rate'].fillna(most_frequent_value, inplace=True)


most_frequent_value = test['host_response_rate'].mode()[0]
test['host_response_rate'].fillna(most_frequent_value, inplace=True)


most_frequent_valuee = test['host_response_time'].mode()[0]
test['host_response_time'].fillna(most_frequent_valuee, inplace=True)

most_frequent_valuee = train['host_response_time'].mode()[0]
train['host_response_time'].fillna(most_frequent_valuee, inplace=True)


## tuened response rate from percentage to rate
train.host_response_rate = train.host_response_rate.str.replace('%', '').str.replace(',' , '').astype(float)
test.host_response_rate = test.host_response_rate.str.replace('%', '').str.replace(',' , '').astype(float)






## filled in any missing values with median as data was skewed and numeric
train.host_response_rate = train.host_response_rate.fillna(train.host_response_rate.median())
test.host_response_rate = test.host_response_rate.fillna(test.host_response_rate.median())



train.host_acceptance_rate = train.host_acceptance_rate.fillna(train.host_response_rate.median())
test.host_acceptance_rate = test.host_acceptance_rate.fillna(test.host_response_rate.median())

train.review_scores_rating = train.review_scores_rating.fillna(train.review_scores_rating.median())
test.review_scores_rating = test.review_scores_rating.fillna(test.review_scores_rating.median())

train.review_scores_accuracy = train.review_scores_accuracy.fillna(train.review_scores_accuracy.median())
test.review_scores_accuracy = test.review_scores_accuracy.fillna(test.review_scores_accuracy.median())


train.review_scores_cleanliness = train.review_scores_cleanliness.fillna(train.review_scores_cleanliness.median())
test.review_scores_cleanliness = test.review_scores_cleanliness.fillna(test.review_scores_cleanliness.median())

train.review_scores_checkin = train.review_scores_checkin.fillna(train.review_scores_checkin.median())
test.review_scores_checkin = test.review_scores_checkin.fillna(test.review_scores_checkin.median())

train.review_scores_communication = train.review_scores_communication.fillna(train.review_scores_communication.median())
test.review_scores_communication = test.review_scores_communication.fillna(test.review_scores_communication.median())


#filled in any missing values with at least one 
test['bathrooms_text'] = test['bathrooms_text'].fillna('1 bath')

train['bathrooms_text'] = train['bathrooms_text'].fillna('1 bath')

train['year'] = pd.DatetimeIndex(train['host_since']).year
test['year'] = pd.DatetimeIndex(test['host_since']).year


import math

def log_review(x):
    if x == 0:
        return 0
    else:
        return math.log(x)

train['reviews_log'] = train.number_of_reviews.apply(log_review)
test['reviews_log'] = test.number_of_reviews.apply(log_review)


## began dealing with bathrooms_text, distinguished those with shared bathrooms 
train['shared_bathroom'] = train['bathrooms_text'].str.contains('shared', case=False, na=False).astype(int)
test['shared_bathroom'] = test['bathrooms_text'].str.contains('shared', case=False, na=False).astype(int)



train['instant_bookable'] = train['instant_bookable'].map({'f': 0, 't': 1})
test['instant_bookable'] = test['instant_bookable'].map({'f': 0, 't': 1})

test['has_availability'] = test['has_availability'].map({'f': 0, 't': 1})
train['has_availability'] = train['has_availability'].map({'f': 0, 't': 1})


test['host_identity_verified'] = test['host_identity_verified'].map({'f': 0, 't': 1})
train['host_identity_verified'] = train['host_identity_verified'].map({'f': 0, 't': 1})




test['host_has_profile_pic'] = test['host_has_profile_pic'].map({'f': 0, 't': 1})
train['host_has_profile_pic'] = train['host_has_profile_pic'].map({'f': 0, 't': 1})


train.host_acceptance_rate = train.host_acceptance_rate.str.replace('%', '').str.replace(',' , '').astype(float)
test.host_acceptance_rate = test.host_acceptance_rate.str.replace('%', '').str.replace(',' , '').astype(float)


train.host_acceptance_rate = train.host_acceptance_rate.fillna(train.host_acceptance_rate.median())
test.host_acceptance_rate = test.host_acceptance_rate.fillna(test.host_acceptance_rate.median())



test['reviews_per_month'] = test['reviews_per_month'].fillna("0")

train['reviews_per_month'] = train['reviews_per_month'].fillna("0")



test['host_neighbourhood'] = test['host_neighbourhood'].fillna("Other")

train['host_neighbourhood'] = train['host_neighbourhood'].fillna("Other")

test['host_location'] = test['host_location'].fillna("Other")

train['host_location'] = train['host_location'].fillna("Other")




top_n = 3  
top_categories = train['host_location'].value_counts().index[:top_n]

def bin_host_location(location):
    if location in top_categories:
        return location
    else:
        return 'Other'

train['host_location_binned'] = train['host_location'].apply(bin_host_location)
test['host_location_binned'] = test['host_location'].apply(bin_host_location)




top_n = 3  
top_categories = train['host_neighbourhood'].value_counts().index[:top_n]

def bin_host_neighbourhood(neighbourhood):
    if neighbourhood in top_categories:
        return neighbourhood
    else:
        return 'Other'

train['host_neighbourhood_binned'] = train['host_neighbourhood'].apply(bin_host_neighbourhood)
test['host_neighbourhood_binned'] = test['host_neighbourhood'].apply(bin_host_neighbourhood)



top_n = 3 
top_categories = train['property_type'].value_counts().index[:top_n]

def bin_property_type(property):
    if property in top_categories:
        return property
    else:
        return 'Other'

train['property_binned'] = train['property_type'].apply(bin_property_type)
test['property_binned'] = test['property_type'].apply(bin_property_type)



top_n = 3 
top_categories = train['neighbourhood_cleansed'].value_counts().index[:top_n]

def bin_neighbourhood_cleansed(neighbourhood):
    if neighbourhood in top_categories:
        return neighbourhood
    else:
        return 'Other'

train['neighbourhood_cleansed_binned'] = train['neighbourhood_cleansed'].apply(bin_neighbourhood_cleansed)
test['neighbourhood_cleansed_binned'] = test['neighbourhood_cleansed'].apply(bin_neighbourhood_cleansed)


/tmp/ipykernel_172694/3453720696.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['host_is_superhost'].fillna(most_frequent_value, inplace=True)
/tmp/ipykernel_172694/3453720696.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['host_is_superhost'] = train['host_is_superhost'].repla

In [4]:
predictors = ['host_response_time', 'host_response_rate',
       'host_acceptance_rate',  
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed_binned', 'latitude', 'longitude', 'property_binned',
       'room_type', 'accommodates', 'beds', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'number_of_reviews_ltm',
       'number_of_reviews_l30d', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month','year', 'reviews_log',
             'host_neighbourhood_binned', 'host_location_binned']



X_train = train[predictors]
y_train = train['host_is_superhost']
X_test = test[predictors]

In [5]:
X_test.host_verifications

X_test['host_verifications'] = X_test['host_verifications'].str.replace("[", "").str.replace("]", "").str.replace("'", "")

X_train.host_verifications

X_train['host_verifications'] = X_train['host_verifications'].str.replace("[", "").str.replace("]", "").str.replace("'", "")


relevant_columns = X_train.loc[:,["host_has_profile_pic", 
                 "host_identity_verified" , 
                                  'neighbourhood_cleansed_binned' , 'property_binned', 'room_type', 'host_neighbourhood_binned', 'host_location_binned', 'host_response_time', 'host_verifications']]
train_clean = pd.get_dummies(relevant_columns, drop_first = True)
train_clean
X_train_total = pd.concat([X_train.drop(relevant_columns.columns, axis=1), train_clean], axis=1)

relevant_columns = X_test.loc[:,[ "host_has_profile_pic", 
                 "host_identity_verified" , 
                                  'neighbourhood_cleansed_binned' , 'property_binned', 'room_type', 'host_neighbourhood_binned', 'host_location_binned', 'host_response_time', 'host_verifications']]
test_clean = pd.get_dummies(relevant_columns, drop_first = True)
test_clean
X_test_total = pd.concat([X_test.drop(relevant_columns.columns, axis=1), test_clean], axis=1)

X_train_total.reviews_per_month = X_train_total.reviews_per_month.astype(float)
X_test_total.reviews_per_month = X_test_total.reviews_per_month.astype(float)

relevant_columns = X_train.loc[:,["host_has_profile_pic", 
                 "host_identity_verified" , 
                                  'neighbourhood_cleansed_binned' , 'property_binned', 'room_type', 'host_neighbourhood_binned', 'host_location_binned', 'host_response_time', 'host_verifications']]
train_clean = pd.get_dummies(relevant_columns, drop_first = True)
train_clean
X_train_total = pd.concat([X_train.drop(relevant_columns.columns, axis=1), train_clean], axis=1)

relevant_columns = X_test.loc[:,[ "host_has_profile_pic", 
                 "host_identity_verified" , 
                                  'neighbourhood_cleansed_binned' , 'property_binned', 'room_type', 'host_neighbourhood_binned', 'host_location_binned', 'host_response_time', 'host_verifications']]
test_clean = pd.get_dummies(relevant_columns, drop_first = True)
test_clean
X_test_total = pd.concat([X_test.drop(relevant_columns.columns, axis=1), test_clean], axis=1)
X_train_total.reviews_per_month = X_train_total.reviews_per_month.astype(float)
X_test_total.reviews_per_month = X_test_total.reviews_per_month.astype(float)
X_test_total = X_test_total.replace({True: 1, False:0})
X_train_total = X_train_total.replace({True:1 , False: 0})

/tmp/ipykernel_172694/3205076460.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['host_verifications'] = X_test['host_verifications'].str.replace("[", "").str.replace("]", "").str.replace("'", "")
/tmp/ipykernel_172694/3205076460.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['host_verifications'] = X_train['host_verifications'].str.replace("[", "").str.replace("]", "").str.replace("'", "")
/tmp/ipykernel_172694/3205076460.py:42: FutureWarning: Downcasting behavior in `replace` is de

In [13]:
import re
X_train_total = X_train_total.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_test_total = X_test_total.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [14]:
cat= CatBoostClassifier(random_state = 1, learning_rate = 0.1, max_depth =9, 
                      n_estimators = 300, reg_lambda = 0.1, subsample = 0.5).fit(X_train_total, y_train)

boost = LGBMClassifier(random_state=1,
                       colsample_bytree = 0.5, 
                       learning_rate = 0.01, 
                       max_depth = 15, 
                       n_estimators = 5250, 
                       reg_lambda = 0, 
                       subsample = 0.5,
                       scale_pos_weight = 1.279, verbose = -1, num_threads = 1).fit(X_train_total, y_train)

xgb = XGBClassifier(random_state=1, scale_pos_weight = 1.279, 
                   gamma = 0.01, learning_rate = 0.1, max_depth = 8, n_estimators = 700, reg_lambda= 0).fit(X_train_total,y_train)


0:	learn: 0.6131754	total: 19.6ms	remaining: 5.86s
1:	learn: 0.5592403	total: 39.1ms	remaining: 5.83s
2:	learn: 0.5191977	total: 58ms	remaining: 5.75s
3:	learn: 0.4897438	total: 76.9ms	remaining: 5.69s
4:	learn: 0.4635340	total: 94.8ms	remaining: 5.59s
5:	learn: 0.4446270	total: 112ms	remaining: 5.5s
6:	learn: 0.4183747	total: 130ms	remaining: 5.45s
7:	learn: 0.3975642	total: 148ms	remaining: 5.39s
8:	learn: 0.3736892	total: 164ms	remaining: 5.32s
9:	learn: 0.3563546	total: 182ms	remaining: 5.28s
10:	learn: 0.3461802	total: 200ms	remaining: 5.26s
11:	learn: 0.3327583	total: 218ms	remaining: 5.22s
12:	learn: 0.3238464	total: 238ms	remaining: 5.24s
13:	learn: 0.3103623	total: 256ms	remaining: 5.23s
14:	learn: 0.3006006	total: 275ms	remaining: 5.23s
15:	learn: 0.2916775	total: 293ms	remaining: 5.19s
16:	learn: 0.2849347	total: 311ms	remaining: 5.18s
17:	learn: 0.2742971	total: 331ms	remaining: 5.18s
18:	learn: 0.2657522	total: 349ms	remaining: 5.16s
19:	learn: 0.2572216	total: 366ms	remai

167:	learn: 0.0140556	total: 3.02s	remaining: 2.37s
168:	learn: 0.0138213	total: 3.04s	remaining: 2.35s
169:	learn: 0.0136063	total: 3.06s	remaining: 2.34s
170:	learn: 0.0134682	total: 3.08s	remaining: 2.32s
171:	learn: 0.0133174	total: 3.09s	remaining: 2.3s
172:	learn: 0.0130939	total: 3.11s	remaining: 2.28s
173:	learn: 0.0129322	total: 3.13s	remaining: 2.27s
174:	learn: 0.0127356	total: 3.15s	remaining: 2.25s
175:	learn: 0.0124412	total: 3.17s	remaining: 2.23s
176:	learn: 0.0122536	total: 3.18s	remaining: 2.21s
177:	learn: 0.0120952	total: 3.2s	remaining: 2.19s
178:	learn: 0.0118846	total: 3.22s	remaining: 2.18s
179:	learn: 0.0117420	total: 3.24s	remaining: 2.16s
180:	learn: 0.0115534	total: 3.26s	remaining: 2.14s
181:	learn: 0.0113791	total: 3.28s	remaining: 2.12s
182:	learn: 0.0112372	total: 3.29s	remaining: 2.11s
183:	learn: 0.0109778	total: 3.31s	remaining: 2.09s
184:	learn: 0.0108208	total: 3.33s	remaining: 2.07s
185:	learn: 0.0106516	total: 3.35s	remaining: 2.05s
186:	learn: 0.

In [15]:
ensemble_model = VotingClassifier(estimators=[('lgbm', boost),('xg',xgb), ('cat', cat)])
ensemble_model.fit(X_train_total,y_train)

0:	learn: 0.6131754	total: 15.1ms	remaining: 4.51s
1:	learn: 0.5592403	total: 33.9ms	remaining: 5.04s
2:	learn: 0.5191977	total: 52.1ms	remaining: 5.15s
3:	learn: 0.4897438	total: 69.9ms	remaining: 5.17s
4:	learn: 0.4635340	total: 87.6ms	remaining: 5.17s
5:	learn: 0.4446270	total: 105ms	remaining: 5.14s
6:	learn: 0.4183747	total: 121ms	remaining: 5.08s
7:	learn: 0.3975642	total: 139ms	remaining: 5.08s
8:	learn: 0.3736892	total: 157ms	remaining: 5.07s
9:	learn: 0.3563546	total: 175ms	remaining: 5.09s
10:	learn: 0.3461802	total: 193ms	remaining: 5.08s
11:	learn: 0.3327583	total: 212ms	remaining: 5.09s
12:	learn: 0.3238464	total: 230ms	remaining: 5.08s
13:	learn: 0.3103623	total: 248ms	remaining: 5.06s
14:	learn: 0.3006006	total: 266ms	remaining: 5.06s
15:	learn: 0.2916775	total: 283ms	remaining: 5.02s
16:	learn: 0.2849347	total: 301ms	remaining: 5.01s
17:	learn: 0.2742971	total: 320ms	remaining: 5.02s
18:	learn: 0.2657522	total: 337ms	remaining: 4.99s
19:	learn: 0.2572216	total: 356ms	re

164:	learn: 0.0147553	total: 2.96s	remaining: 2.43s
165:	learn: 0.0145030	total: 2.98s	remaining: 2.41s
166:	learn: 0.0141843	total: 3s	remaining: 2.39s
167:	learn: 0.0140556	total: 3.02s	remaining: 2.37s
168:	learn: 0.0138213	total: 3.04s	remaining: 2.35s
169:	learn: 0.0136063	total: 3.05s	remaining: 2.33s
170:	learn: 0.0134682	total: 3.07s	remaining: 2.32s
171:	learn: 0.0133174	total: 3.09s	remaining: 2.3s
172:	learn: 0.0130939	total: 3.11s	remaining: 2.28s
173:	learn: 0.0129322	total: 3.13s	remaining: 2.26s
174:	learn: 0.0127356	total: 3.14s	remaining: 2.25s
175:	learn: 0.0124412	total: 3.16s	remaining: 2.23s
176:	learn: 0.0122536	total: 3.18s	remaining: 2.21s
177:	learn: 0.0120952	total: 3.2s	remaining: 2.19s
178:	learn: 0.0118846	total: 3.22s	remaining: 2.17s
179:	learn: 0.0117420	total: 3.23s	remaining: 2.16s
180:	learn: 0.0115534	total: 3.25s	remaining: 2.14s
181:	learn: 0.0113791	total: 3.27s	remaining: 2.12s
182:	learn: 0.0112372	total: 3.29s	remaining: 2.1s
183:	learn: 0.0109

VotingClassifier(estimators=[('lgbm',
                              LGBMClassifier(colsample_bytree=0.5,
                                             learning_rate=0.01, max_depth=15,
                                             n_estimators=5250, num_threads=1,
                                             random_state=1, reg_lambda=0,
                                             scale_pos_weight=1.279,
                                             subsample=0.5, verbose=-1)),
                             ('xg',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, devi...
                                            interaction_constraints=None,
                                            learning_rate=0.1, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=8,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=700, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=1, ...)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x2b08bfb671d0>)])

In [16]:
y_pred = ensemble_model.predict(X_test_total)


output = pd.DataFrame({'id':test.id, 'predicted':y_pred})
output

output.to_csv("submission.csv", index= False)